# Demonstrate Classification

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from CONFIG import PREPROCESSED_DATA_PATH
from knn import KNN
from gensim.models import Word2Vec
import gensim.downloader
from classifier import Classifier
from word2vec import Word2VecModel
import math
import sklearn

In [2]:
# Set up some configs
SHOULD_RETRAIN_WORD2VEC = False
WORD2VEC_PATH = "custom_word_2_vec"

In [3]:
# Load data and shuffle
df_data = pd.read_parquet(PREPROCESSED_DATA_PATH)
df_data = df_data.sample(frac=1).reset_index(drop=True)

In [4]:
if SHOULD_RETRAIN_WORD2VEC:
    # Configure data
    dataset_size_limt = 100000 #len(df_data)#10000
    df_data = df_data[:dataset_size_limt]
    test_subset = df_data[:int(len(df_data)*0.2)]
    train_subset = df_data[int(len(df_data)*0.2):]
    
    # Fit the word2vec model
    training_tokens = np.array([x.split() for x in train_subset["title"]])
    w2v_custom = Word2VecModel()
    w2v_custom.fit(training_tokens, epochs=2048, batch_size=8192)
    
    # Save the word 2 vec model
    w2v_custom.save(WORD2VEC_PATH)

## Set up Classifier

In [15]:
c = Classifier(WORD2VEC_PATH)

In [16]:
df_data = df_data.sample(frac=1).reset_index(drop=True)
classes = np.array(df_data["categories"])
classes = classes[:10000]
titles = np.array(df_data["title"])
titles = titles[:10000]

In [17]:
c.fit(titles, classes)

# K-Fold Validation

In [8]:
SHOULD_RUN_K_FOLD = False

In [9]:
if SHOULD_RUN_K_FOLD:
    kf = KFold(n_splits=10)
    i = 0
    for train_index, test_index in kf.split(titles):
        print("FOLD ", i)
        X_train, X_test = titles[train_index], titles[test_index]
        y_train, y_test = classes[train_index], classes[test_index]
        c.fit(X_train, y_train)

        predictions = c.predict(X_test)
        f_score = sklearn.metrics.f1_score(y_test, predictions, average='macro')
        precision = sklearn.metrics.precision_score(y_test, predictions, average='macro')
        recall = sklearn.metrics.recall_score(y_test, predictions, average='macro')
        acc = sklearn.metrics.accuracy_score(y_test, predictions)
        print("F Score:", f_score, "Precision", precision, "Recall", recall, "Accuracy", acc)

## Classify Titles

In [18]:
test = ["apple iphone 5 new 4g lte for all carrier red textured skin"]
#print(c.predict(test)[0])

cellphonesaccessories


In [11]:
test = ["iphone cable"]
#print(c.predict(test)[0])

In [12]:
test = ["star trek"]
#print(c.predict(test)[0])

In [13]:
test = ["oliver twist"]
#print(c.predict(test)[0])

In [14]:
test = ["dish set"]
#print(c.predict(test)[0])